In [22]:
import pandas as pd
from selenium import webdriver
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
from time import sleep
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
dir_for_shinhan = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'

In [4]:
header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [11]:
pd.options.display.float_format = '{:.0f}'.format

In [14]:
region_data = pd.read_csv('C:/Users/shic/Desktop/crawler_project/data_save/kb_land/region_code.csv')
region_data = region_data[region_data['small_region_name']!='(전체)']
region_data

,big_region_code,big_region_name,small_region_code,small_region_name
2,1100000000,서울,1199000000,서울강북
3,1100000000,서울,1130500000,서울강북구
4,1100000000,서울,1121500000,서울광진구
5,1100000000,서울,1135000000,서울노원구
6,1100000000,서울,1132000000,서울도봉구
...,...,...,...,...
178,4800000000,경남,4833000000,양산
179,4800000000,경남,4831000000,거제
180,4800000000,경남,4817000000,진주
181,4800000000,경남,4825000000,김해


In [18]:
region_code = region_data[region_data['big_region_name'].isin(['서울','경기','인천'])]

In [33]:
kb_list = []
base_url = 'https://onland.kbstar.com/quics?page=C060737&%EB%B2%95%EC%A0%95%EB%8F%99%EB%8C%80%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C={}&%EB%B2%95%EC%A0%95%EB%8F%99%EC%A4%91%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C={}&%EA%B8%B0%EC%A4%80%EB%85%84={}&%EA%B8%B0%EC%A4%80%EC%9B%94={}&QSL=F'

for year in range(2010,2022):
    year = str(year)
    for month in range(1,13):
        if int(month) < 10:
            month = '0' + str(month)
        else:
            month = str(month)
        for region in region_code.iterrows():
            # format은 대지역 코드, 중지역 코드, 년, 월 순.
            tmp_url = base_url.format(int(region[1]['big_region_code']), #대지역코드
                                      int(region[1]['small_region_code']), #중지역코드
                                      year,month)
            
            # URL 접속
            tmp_req = requests.get(tmp_url,headers=header)
            
            # 그래프의 데이터 가져오기
            soup = BeautifulSoup(tmp_req.text,'html.parser')
            
            aa = re.search('var graphYn1 = [$](.+?);',tmp_req.text,re.S).group(1)
            aa = aa.replace(".trim('[","").replace("') == '' ? false: true","").replace('Date.UTC(','')
            aa = aa.split('], [')
            
            date_list_aa = []
            data_list_aa = []
            
            for k in aa:
                tmpp = k.split('),')
                data_wrangle = tmpp[0].split(', ')
                data_wrangle[1] = str(int(data_wrangle[1]) + 1)
                
                date_list_aa.append(','.join(data_wrangle))
                data_list_aa.append(float(tmpp[1].replace(']','')))
                
            bb = re.search('var graphYn2 = [$](.+?);',tmp_req.text,re.S).group(1)
            bb = bb.replace(".trim('[","").replace("') == '' ? false: true","").replace('Date.UTC(','')
            bb = bb.split('], [')
            
            date_list_bb = []
            data_list_bb = []
            
            for k in bb:
                tmpp = k.split('),')
                data_wrangle = tmpp[0].split(', ')
                data_wrangle[1] = str(int(data_wrangle[1]) + 1)
                
                date_list_bb.append(','.join(data_wrangle))
                data_list_bb.append(float(tmpp[1].replace(']','').replace(',','')))
                
            kb_list.append(pd.DataFrame({
                'b_region' : region[1]['big_region_name'],
                's_region' : region[1]['small_region_name'],
                'date_a'   : pd.to_datetime(date_list_aa,format='%Y,%m,%d'),
                'value1'   : data_list_aa,
                'date_b'   : pd.to_datetime(date_list_bb,format='%Y,%m,%d'),
                'value2'   : data_list_bb
            }))
            
            # 일단 2021년 6월 까지의 데이터만 가져온다.
            if year == '2021' and month == '06':
                break

In [36]:
df = pd.concat(kb_list,axis=0).drop_duplicates()
df['date_a'] = pd.to_datetime(df['date_a'],format='%Y-%m-%d')
df['date_b'] = pd.to_datetime(df['date_b'],format='%Y-%m-%d')

In [37]:
dirr = "C:/Users/shic/Desktop/crawler_project/data_save/"

In [38]:
df.to_csv(dirr+'kb_land_tmp.csv',index=False)